# Will it rain tomorrow prediction

In [1]:
import numpy as np
import pandas as pd


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [2]:
#loading dataset
df = pd.read_csv("weatherAUS.csv")
df.shape

(142193, 24)

In [3]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [4]:
df.count().sort_values()

Sunshine          74377
Evaporation       81350
Cloud3pm          85099
Cloud9am          88536
Pressure9am      128179
Pressure3pm      128212
WindDir9am       132180
WindGustDir      132863
WindGustSpeed    132923
WindDir3pm       138415
Humidity3pm      138583
Temp3pm          139467
WindSpeed3pm     139563
Humidity9am      140419
RainToday        140787
Rainfall         140787
WindSpeed9am     140845
Temp9am          141289
MinTemp          141556
MaxTemp          141871
Date             142193
Location         142193
RISK_MM          142193
RainTomorrow     142193
dtype: int64

In [5]:
# drop unnecessary columns due to sparsity and no relation
df.drop(columns=['Sunshine','Evaporation','Cloud3pm','Cloud9am','RISK_MM','Location','Date'],inplace=True,axis=1)

In [6]:
df.shape

(142193, 17)

In [7]:
df.isnull().sum()

MinTemp            637
MaxTemp            322
Rainfall          1406
WindGustDir       9330
WindGustSpeed     9270
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Temp9am            904
Temp3pm           2726
RainToday         1406
RainTomorrow         0
dtype: int64

In [8]:
# dropping null values
df.dropna(inplace=True)
df.shape

(112925, 17)

In [9]:
# changing categorial data to suitable one
df['RainToday'].replace({'No':0,'Yes':1},inplace=True)
df['RainTomorrow'].replace({'No':0,'Yes':1},inplace=True)

categories = ['WindGustDir','WindDir9am','WindDir3pm']
for i in categories:
    print(np.unique(df[i]))
df = pd.get_dummies(df,columns=categories)
df.iloc[4:9]

['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']
['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']
['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']


,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,...,0,1,0,0,0,0,0,0,0,0
5,14.6,29.7,0.2,56.0,19.0,24.0,55.0,23.0,1009.2,1005.4,...,0,0,0,0,0,0,0,1,0,0
6,14.3,25.0,0.0,50.0,20.0,24.0,49.0,19.0,1009.6,1008.2,...,0,0,0,0,0,0,0,1,0,0
7,7.7,26.7,0.0,35.0,6.0,17.0,48.0,19.0,1013.4,1010.1,...,0,0,0,0,0,0,0,1,0,0
8,9.7,31.9,0.0,80.0,7.0,28.0,42.0,9.0,1008.9,1003.6,...,0,1,0,0,0,0,0,0,0,0


In [10]:
# preprocessing
scaler = MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df),index=df.index,columns=df.columns)
df.iloc[2:7]

/home/jaipal/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
2,0.501188,0.507692,0.000000,0.304688,0.200000,0.282353,0.38,0.30,0.447934,0.5056,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.413302,0.558242,0.000000,0.132812,0.105882,0.082353,0.45,0.16,0.613223,0.5712,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.610451,0.652747,0.002720,0.265625,0.058824,0.211765,0.82,0.33,0.500826,0.4624,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.541568,0.595604,0.000544,0.382812,0.200000,0.258824,0.55,0.23,0.474380,0.4528,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0.534442,0.492308,0.000000,0.335938,0.211765,0.258824,0.49,0.19,0.480992,0.4976,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
#selecting best features using SelectKBest and chi-square
X = df.loc[:,df.columns != 'RainTomorrow']
y = df['RainTomorrow']
selector = SelectKBest(chi2,k=4)
selector.fit(X,y)
X_new = selector.transform(X)
print(X.columns[selector.get_support(indices=True)])

Index(['Rainfall', 'Humidity9am', 'Humidity3pm', 'RainToday'], dtype='object')


In [12]:
#choose only best columns for fast processing
df = df[['Rainfall', 'Humidity9am', 'Humidity3pm', 'RainToday',"RainTomorrow"]]
X = df.loc[:,df.columns != 'RainTomorrow']
y = df[['RainTomorrow']]

In [13]:
#split data 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [14]:
### Using Logistic Regression
lr_model = LogisticRegression(solver='lbfgs') #solver is an optimization algorithm
lr_model.fit(X_train,y_train)
predictions = lr_model.predict(X_test)
#show results
df = pd.DataFrame(confusion_matrix(y_test,predictions.reshape(-1,1)),index=['Yes','No'],columns=['Yes','No'])
print(df,'\n')
print(classification_report(y_test,predictions))
print(accuracy_score(y_test,predictions))

/home/jaipal/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


       Yes    No
Yes  27590  1284
No    4935  3457 

              precision    recall  f1-score   support

         0.0       0.85      0.96      0.90     28874
         1.0       0.73      0.41      0.53      8392

   micro avg       0.83      0.83      0.83     37266
   macro avg       0.79      0.68      0.71     37266
weighted avg       0.82      0.83      0.81     37266

0.8331186604411528


In [15]:
# Using MultiNomialNB
multi_model = MultinomialNB()#suitable for discrete and occurrance count
multi_model.fit(X_train,y_train)
predictions = multi_model.predict(X_test)
#show results
df = pd.DataFrame(confusion_matrix(y_test,predictions.reshape(-1,1)),index=['Yes','No'],columns=['Yes','No'])
print(df,'\n')
print(classification_report(y_test,predictions))
print(accuracy_score(y_test,predictions))

/home/jaipal/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


       Yes  No
Yes  28874   0
No    8391   1 

              precision    recall  f1-score   support

         0.0       0.77      1.00      0.87     28874
         1.0       1.00      0.00      0.00      8392

   micro avg       0.77      0.77      0.77     37266
   macro avg       0.89      0.50      0.44     37266
weighted avg       0.83      0.77      0.68     37266

0.7748349702141362


In [16]:
# Using BernoulliNB
nb_model = BernoulliNB() #for discrete and binary 
nb_model.fit(X_train,y_train)
predictions = nb_model.predict(X_test)
#show results
df = pd.DataFrame(confusion_matrix(y_test,predictions.reshape(-1,1)),index=['Yes','No'],columns=['Yes','No'])
print(df,'\n')
print(classification_report(y_test,predictions))
print(accuracy_score(y_test,predictions))

       Yes    No
Yes  24369  4505
No    4452  3940 

              precision    recall  f1-score   support

         0.0       0.85      0.84      0.84     28874
         1.0       0.47      0.47      0.47      8392

   micro avg       0.76      0.76      0.76     37266
   macro avg       0.66      0.66      0.66     37266
weighted avg       0.76      0.76      0.76     37266

0.7596468630923631


/home/jaipal/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
decision_model = DecisionTreeClassifier()
decision_model.fit(X_train,y_train)
predictions = decision_model.predict(X_test)
#show results
df = pd.DataFrame(confusion_matrix(y_test,predictions.reshape(-1,1)),index=['Yes','No'],columns=['Yes','No'])
print(df,'\n')
print(classification_report(y_test,predictions))
print(accuracy_score(y_test,predictions))

       Yes    No
Yes  26456  2418
No    5088  3304 

              precision    recall  f1-score   support

         0.0       0.84      0.92      0.88     28874
         1.0       0.58      0.39      0.47      8392

   micro avg       0.80      0.80      0.80     37266
   macro avg       0.71      0.65      0.67     37266
weighted avg       0.78      0.80      0.78     37266

0.7985831589116085


In [22]:
svc_model = SVC()
svc_model.fit(X_train,y_train)
predictions = svc_model.predict(X_test)
#show results
df = pd.DataFrame(confusion_matrix(y_test,predictions.reshape(-1,1)),index=['Yes','No'],columns=['Yes','No'])
print(df,'\n')
print(classification_report(y_test,predictions))
print(accuracy_score(y_test,predictions))

/home/jaipal/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jaipal/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


       Yes    No
Yes  28077   797
No    6029  2363 

              precision    recall  f1-score   support

         0.0       0.82      0.97      0.89     28874
         1.0       0.75      0.28      0.41      8392

   micro avg       0.82      0.82      0.82     37266
   macro avg       0.79      0.63      0.65     37266
weighted avg       0.81      0.82      0.78     37266

0.8168303547469543


In [19]:
linear_svc_model = LinearSVC()
linear_svc_model.fit(X_train,y_train)
predictions = linear_svc_model.predict(X_test)
#show results
df = pd.DataFrame(confusion_matrix(y_test,predictions.reshape(-1,1)),index=['Yes','No'],columns=['Yes','No'])
print(df,'\n')
print(classification_report(y_test,predictions))
print(accuracy_score(y_test,predictions))

/home/jaipal/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


       Yes    No
Yes  27818  1056
No    5217  3175 

              precision    recall  f1-score   support

         0.0       0.84      0.96      0.90     28874
         1.0       0.75      0.38      0.50      8392

   micro avg       0.83      0.83      0.83     37266
   macro avg       0.80      0.67      0.70     37266
weighted avg       0.82      0.83      0.81     37266

0.8316696184189342


In [20]:
#using linear regression as:
lr_model = LinearRegression()
lr_model.fit(X_train,y_train)
predictions = lr_model.predict(X_test)
predictions = np.array([1 if x >= 0.5 else 0 for x in predictions])

#show results

df = pd.DataFrame(confusion_matrix(y_test,predictions.reshape(-1,1)),index=['Yes','No'],columns=['Yes','No'])
print(df,'\n')
print(classification_report(y_test,predictions))
print(accuracy_score(y_test,predictions))

       Yes    No
Yes  27917   957
No    5487  2905 

              precision    recall  f1-score   support

         0.0       0.84      0.97      0.90     28874
         1.0       0.75      0.35      0.47      8392

   micro avg       0.83      0.83      0.83     37266
   macro avg       0.79      0.66      0.69     37266
weighted avg       0.82      0.83      0.80     37266

0.8270809853485751


In [21]:
# using RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
predictions = rfc.predict(X_test)

#show results
df = pd.DataFrame(confusion_matrix(y_test,predictions.reshape(-1,1)),index=['Yes','No'],columns=['Yes','No'])
print(df,'\n')
print(classification_report(y_test,predictions))
print(accuracy_score(y_test,predictions))

/home/jaipal/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jaipal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


       Yes    No
Yes  26739  2135
No    4971  3421 

              precision    recall  f1-score   support

         0.0       0.84      0.93      0.88     28874
         1.0       0.62      0.41      0.49      8392

   micro avg       0.81      0.81      0.81     37266
   macro avg       0.73      0.67      0.69     37266
weighted avg       0.79      0.81      0.79     37266

0.8093168035206354
